In [8]:
import torch
import sys
import numpy as np
sys.path.append('../')

# from utils import intersection_over_union

In [74]:
class anchors () :
    def __init__(self) :
        self.grid_size = 5
        self.stride = 32
        self.kernel_size = 128
#         self.anchors = torch.zeros([self.grid_size, self.grid_size,4])
        self.anchors = np.zeros([self.grid_size, self.grid_size,4])
anchor = anchors()   
anchor.anchors.shape

(5, 5, 4)

In [75]:
# anchors = torch.zeros([self.grid_size, self.grid_size,4])

for x in range(anchor.grid_size) :         
    for y in range(anchor.grid_size) :
        anchor.anchors[x][y][1] = anchor.stride * x
        anchor.anchors[x][y][3] = anchor.stride * x + anchor.kernel_size   
        anchor.anchors[x][y][0] = anchor.stride * y
        anchor.anchors[x][y][2] = anchor.stride * y + anchor.kernel_size

In [76]:
anchor.anchors

array([[[  0.,   0., 128., 128.],
        [ 32.,   0., 160., 128.],
        [ 64.,   0., 192., 128.],
        [ 96.,   0., 224., 128.],
        [128.,   0., 256., 128.]],

       [[  0.,  32., 128., 160.],
        [ 32.,  32., 160., 160.],
        [ 64.,  32., 192., 160.],
        [ 96.,  32., 224., 160.],
        [128.,  32., 256., 160.]],

       [[  0.,  64., 128., 192.],
        [ 32.,  64., 160., 192.],
        [ 64.,  64., 192., 192.],
        [ 96.,  64., 224., 192.],
        [128.,  64., 256., 192.]],

       [[  0.,  96., 128., 224.],
        [ 32.,  96., 160., 224.],
        [ 64.,  96., 192., 224.],
        [ 96.,  96., 224., 224.],
        [128.,  96., 256., 224.]],

       [[  0., 128., 128., 256.],
        [ 32., 128., 160., 256.],
        [ 64., 128., 192., 256.],
        [ 96., 128., 224., 256.],
        [128., 128., 256., 256.]]])

In [12]:
bbox = [[62, 140, 180, 300], [127,95,250,220]]

In [13]:
def batch_iou(boxes, box):
    """Compute the Intersection-Over-Union of a batch of boxes with another
    box.
    Args:
    box1: 2D array of [cx, cy, width, height].
    box2: a single array of [cx, cy, width, height]
    Returns:
    ious: array of a float number in range [0, 1].
    """
    lr = np.maximum(
        np.minimum(boxes[:,0]+0.5*boxes[:,2], box[0]+0.5*box[2]) - \
        np.maximum(boxes[:,0]-0.5*boxes[:,2], box[0]-0.5*box[2]),
        0
    )
    tb = np.maximum(
        np.minimum(boxes[:,1]+0.5*boxes[:,3], box[1]+0.5*box[3]) - \
        np.maximum(boxes[:,1]-0.5*boxes[:,3], box[1]-0.5*box[3]),
        0
    )
    inter = lr*tb
    union = boxes[:,2]*boxes[:,3] + box[2]*box[3] - inter
    return inter/union

In [14]:

batch_iou(anchor.anchors.view(-1,4).numpy(), np.array(bbox[1]))

array([0.07366964, 0.1301093 , 0.1860375 , 0.24146117, 0.29036433,
       0.11646895, 0.20966531, 0.30563256, 0.40449613, 0.49447125,
       0.15787329, 0.28960434, 0.43059146, 0.58184576, 0.72431374,
       0.1947681 , 0.36314526, 0.5495826 , 0.7571505 , 0.9591239 ,
       0.16933571, 0.30717885, 0.45135307, 0.6023048 , 0.7396877 ],
      dtype=float32)

In [15]:
batch_iou(anchor.anchors.view(-1,4).numpy(), np.array(bbox[0]))

array([0.10708443, 0.16157205, 0.20411916, 0.17699316, 0.14894068,
       0.17743771, 0.27319258, 0.3499044 , 0.29556993, 0.2426717 ,
       0.24850641, 0.3906763 , 0.5077658 , 0.41725105, 0.33423012,
       0.32030153, 0.51449764, 0.6792688 , 0.5421601 , 0.42369068,
       0.3725639 , 0.60622466, 0.8074012 , 0.6293608 , 0.48263544],
      dtype=float32)

In [16]:
def batch_iom(boxes, box):
    """Compute the Intersection-Over-Union of a batch of boxes with another
    box.
    Args:
    box1: 2D array of [cx, cy, width, height].
    box2: a single array of [cx, cy, width, height]
    Returns:
    ious: array of a float number in range [0, 1].
    """
    lr = np.maximum(
        np.minimum(boxes[:,0]+0.5*boxes[:,2], box[0]+0.5*box[2]) - \
        np.maximum(boxes[:,0]-0.5*boxes[:,2], box[0]-0.5*box[2]),
        0
    )
    tb = np.maximum(
        np.minimum(boxes[:,1]+0.5*boxes[:,3], box[1]+0.5*box[3]) - \
        np.maximum(boxes[:,1]-0.5*boxes[:,3], box[1]-0.5*box[3]),
        0
    )
    inter = lr*tb
#     union = boxes[:,2]*boxes[:,3] + box[2]*box[3] - inter
    iou_max = boxes[:,2]*boxes[:,3]
    return inter/iou_max

In [17]:
batch_iom(anchor.anchors.view(-1,4).numpy(), np.array(bbox[0]))

array([0.41552734, 0.5058594 , 0.5419922 , 0.43359375, 0.34326172,
       0.5480469 , 0.6671875 , 0.71484375, 0.571875  , 0.45273438,
       0.63639325, 0.77473956, 0.8300781 , 0.6640625 , 0.5257161 ,
       0.69949776, 0.8515625 , 0.9123884 , 0.72991073, 0.577846  ,
       0.71875   , 0.875     , 0.9375    , 0.75      , 0.59375   ],
      dtype=float32)

In [18]:
batch_iom(anchor.anchors.view(-1,4).numpy(), np.array(bbox[0]))

array([0.41552734, 0.5058594 , 0.5419922 , 0.43359375, 0.34326172,
       0.5480469 , 0.6671875 , 0.71484375, 0.571875  , 0.45273438,
       0.63639325, 0.77473956, 0.8300781 , 0.6640625 , 0.5257161 ,
       0.69949776, 0.8515625 , 0.9123884 , 0.72991073, 0.577846  ,
       0.71875   , 0.875     , 0.9375    , 0.75      , 0.59375   ],
      dtype=float32)

In [19]:
def batch_iom1(boxes, box):
    """Compute the Intersection-Over-Union of a batch of boxes with another
    box.
    Args:
    box1: 2D array of [cx, cy, width, height].
    box2: a single array of [cx, cy, width, height]
    Returns:
    ious: array of a float number in range [0, 1].
    """
    lr = np.maximum(
        np.minimum(boxes[:,0]+0.5*boxes[:,2], box[0]+0.5*box[2]) - \
        np.maximum(boxes[:,0]-0.5*boxes[:,2], box[0]-0.5*box[2]),
        0
    )
    tb = np.maximum(
        np.minimum(boxes[:,1]+0.5*boxes[:,3], box[1]+0.5*box[3]) - \
        np.maximum(boxes[:,1]-0.5*boxes[:,3], box[1]-0.5*box[3]),
        0
    )
    inter = lr*tb
#     union = boxes[:,2]*boxes[:,3] + box[2]*box[3] - inter
    iou_max = box[2]*box[3]
    return inter/iou_max

In [20]:
batch_iom1(anchor.anchors.view(-1,4).numpy(), np.array(bbox[0]))

array([0.12607408, 0.19185185, 0.24666667, 0.23022223, 0.2082963 ,
       0.20785186, 0.3162963 , 0.40666667, 0.37955555, 0.34340742,
       0.28962964, 0.44074073, 0.56666666, 0.5288889 , 0.47851852,
       0.37140742, 0.5651852 , 0.7266667 , 0.67822224, 0.61362964,
       0.43614814, 0.6637037 , 0.85333335, 0.7964444 , 0.7205926 ],
      dtype=float32)

In [21]:
anchor.anchors

tensor([[[  0.,   0., 128., 128.],
         [ 32.,   0., 160., 128.],
         [ 64.,   0., 192., 128.],
         [ 96.,   0., 224., 128.],
         [128.,   0., 256., 128.]],

        [[  0.,  32., 128., 160.],
         [ 32.,  32., 160., 160.],
         [ 64.,  32., 192., 160.],
         [ 96.,  32., 224., 160.],
         [128.,  32., 256., 160.]],

        [[  0.,  64., 128., 192.],
         [ 32.,  64., 160., 192.],
         [ 64.,  64., 192., 192.],
         [ 96.,  64., 224., 192.],
         [128.,  64., 256., 192.]],

        [[  0.,  96., 128., 224.],
         [ 32.,  96., 160., 224.],
         [ 64.,  96., 192., 224.],
         [ 96.,  96., 224., 224.],
         [128.,  96., 256., 224.]],

        [[  0., 128., 128., 256.],
         [ 32., 128., 160., 256.],
         [ 64., 128., 192., 256.],
         [ 96., 128., 224., 256.],
         [128., 128., 256., 256.]]])

In [22]:
bbox = [[120, 90, 180, 180], [127,95,250,220], [120,90,260,240]]

In [23]:
batch_iom1(anchor.anchors.view(-1,4).numpy(), np.array(bbox[2]))

array([0.11147436, 0.18378206, 0.25608975, 0.32839742, 0.37660256,
       0.16839744, 0.2776282 , 0.38685897, 0.49608976, 0.56891024,
       0.22532052, 0.37147436, 0.5176282 , 0.66378206, 0.76121795,
       0.26564103, 0.4379487 , 0.61025643, 0.7825641 , 0.8974359 ,
       0.24903846, 0.4105769 , 0.57211536, 0.73365384, 0.84134614],
      dtype=float32)

In [24]:
batch_iom(anchor.anchors.view(-1,4).numpy(), np.array(bbox[2]))

array([0.42456055, 0.55996096, 0.65022784, 0.7147042 , 0.7171631 ,
       0.51308596, 0.6767188 , 0.7858073 , 0.8637277 , 0.8666992 ,
       0.57210284, 0.7545573 , 0.8761936 , 0.96307665, 0.96638995,
       0.578125  , 0.7625    , 0.8854167 , 0.97321427, 0.9765625 ,
       0.47424316, 0.6254883 , 0.72631836, 0.79833984, 0.8010864 ],
      dtype=float32)

In [25]:
batch_iou(anchor.anchors.view(-1,4).numpy(), np.array(bbox[1]))

array([0.07366964, 0.1301093 , 0.1860375 , 0.24146117, 0.29036433,
       0.11646895, 0.20966531, 0.30563256, 0.40449613, 0.49447125,
       0.15787329, 0.28960434, 0.43059146, 0.58184576, 0.72431374,
       0.1947681 , 0.36314526, 0.5495826 , 0.7571505 , 0.9591239 ,
       0.16933571, 0.30717885, 0.45135307, 0.6023048 , 0.7396877 ],
      dtype=float32)

In [63]:
def batch_iou_all(boxes, box):
    """Compute the Intersection-Over-Union of a batch of boxes with another
    box.
    Args:
    box1: 2D array of [cx, cy, width, height].
    box2: a single array of [cx, cy, width, height]
    Returns:
    ious: array of a float number in range [0, 1].
    """
    lr = np.maximum(
        np.minimum(boxes[:,0]+0.5*boxes[:,2], box[0]+0.5*box[2]) - \
        np.maximum(boxes[:,0]-0.5*boxes[:,2], box[0]-0.5*box[2]),
        0
    )
    tb = np.maximum(
        np.minimum(boxes[:,1]+0.5*boxes[:,3], box[1]+0.5*box[3]) - \
        np.maximum(boxes[:,1]-0.5*boxes[:,3], box[1]-0.5*box[3]),
        0
    )
    inter = lr*tb
    union = boxes[:,2]*boxes[:,3] + box[2]*box[3] - inter  # iou case
    iou_max = box[2]*box[3] # anchors include bbox
    iou_max1 = boxes[:,2]*boxes[:,3]  # bbox include anchors
    return np.maximum(inter/union, inter/iou_max, inter/iou_max1)

In [58]:
batch_iou_all(anchor.anchors.view(-1,4).numpy(), np.array(bbox[0]))

array([0.06716049, 0.16197531, 0.25679013, 0.35160494, 0.35555556,
       0.11753087, 0.2834568 , 0.44938272, 0.61530864, 0.62222224,
       0.16790123, 0.40493828, 0.6419753 , 0.87901235, 0.8888889 ,
       0.18888889, 0.45555556, 0.7222222 , 0.98888886, 1.        ,
       0.18888889, 0.45555556, 0.7222222 , 0.98888886, 1.        ],
      dtype=float32)

In [59]:
batch_iou(anchor.anchors.view(-1,4).numpy(), np.array(bbox[0]))

array([0.04668726, 0.11017803, 0.17099638, 0.22930756, 0.21473308,
       0.07760026, 0.18813504, 0.29983526, 0.41271946, 0.37894738,
       0.10555728, 0.2624    , 0.42918456, 0.60688716, 0.5459509 ,
       0.11136992, 0.27599102, 0.4499308 , 0.63400346, 0.56501114,
       0.10364449, 0.25187713, 0.40239373, 0.5552475 , 0.49438477],
      dtype=float32)

In [60]:
batch_iom(anchor.anchors.view(-1,4).numpy(), np.array(bbox[0]))

array([0.1328125 , 0.25625   , 0.33854166, 0.39732143, 0.3515625 ,
       0.1859375 , 0.35875   , 0.47395834, 0.55625   , 0.4921875 ,
       0.22135417, 0.42708334, 0.5642361 , 0.66220236, 0.5859375 ,
       0.21344866, 0.41183037, 0.54408485, 0.6385523 , 0.56501114,
       0.18676758, 0.36035156, 0.47607422, 0.5587333 , 0.49438477],
      dtype=float32)

In [61]:
batch_iom1(anchor.anchors.view(-1,4).numpy(), np.array(bbox[0]))

array([0.06716049, 0.16197531, 0.25679013, 0.35160494, 0.35555556,
       0.11753087, 0.2834568 , 0.44938272, 0.61530864, 0.62222224,
       0.16790123, 0.40493828, 0.6419753 , 0.87901235, 0.8888889 ,
       0.18888889, 0.45555556, 0.7222222 , 0.98888886, 1.        ,
       0.18888889, 0.45555556, 0.7222222 , 0.98888886, 1.        ],
      dtype=float32)

In [68]:
batch_iou_all(anchor.anchors.view(-1,4).numpy(), np.array(bbox[0]))

array([0.06716049, 0.16197531, 0.25679013, 0.35160494, 0.35555556,
       0.11753087, 0.2834568 , 0.44938272, 0.61530864, 0.62222224,
       0.16790123, 0.40493828, 0.6419753 , 0.87901235, 0.8888889 ,
       0.18888889, 0.45555556, 0.7222222 , 0.98888886, 1.        ,
       0.18888889, 0.45555556, 0.7222222 , 0.98888886, 1.        ],
      dtype=float32)